# Домашнее задание №1

На семинаре мы проходили метрики, необходимо реализовать некоторые из них

In [114]:
import numpy as np
import pandas as pd

## Задание 1. Реализовать метрики Recall@k и  Money Recall@k

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

$$\Large Recall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}}{|Rel_i|}$$

$\Large |Rel_i|$ -- количество релевантных товаров для пользователя $i$

$$\Large MoneyRecall@K(i) = \frac {\sum_{j=1}^{K}\mathbb{1}_{r_{ij}}\cdot Price(j)}{\sum_{s\in Rel_i}Price(s)}$$


In [115]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #TODO: Ваш код здесь
    recommended_list = recommended_list[:k] #cut recommended list to k
    flags = np.isin(bought_list, recommended_list) #which of the purchases were in the Rec.List
    recall = flags.sum() / len(bought_list)#num of purchases from Rec.List divide by all purchases => recall
    
    return recall #Добавьте сюда результат расчета метрики


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)
    
    recommended_list = recommended_list[:k] #cut Rec.List and Rec.Prices to k
    prices_recommended = prices_recommended[:k]
    
    flags = np.isin(bought_list, recommended_list) #which of the purchases were in Rec.List
    recall = (flags * prices_bought).sum() / prices_bought.sum() #cost of purchases from Rec.List divide by total money spent
    #TODO: Ваш код здесь
    
    return recall #Добавьте сюда результат расчета метрики

## Задание 2. Реализовать метрику MRR@k

Mean Reciprocal Rank

- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $\Large rank_j$
- Посчитать reciprocal rank = $\Large\frac{1}{rank_j}$

$$\Large  MMR(i)@k=\frac {1}{\min\limits_{j\in Rel(i)} rank_j}$$

In [116]:
def mrr_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #TODO: Ваш код здесь
    #which of the purchases were in Rec.List
    flags = list(np.isin(recommended_list[:k], bought_list))
    
    #rank of the 1st relevant recommendation if exists
    rank = (flags.index(True)+1) if True in flags else 0
    
    #mrr if rank exists
    mrr = 1/rank if rank else 0
    
    
    return mrr#Добавьте сюда результат расчета метрики

In [117]:
a = [0,0,1,4,5]
b = [1,2,3]
mrr_at_k(a,b)

0.3333333333333333

In [118]:
data = pd.DataFrame({"user_id": ["u1","u2","u3"], 
              "recommended_list": [
                                      [143, 156, 1134, 991, 27],
                                      [1543, 3345, 533, 11, 43],
                                      [156, 3345, 10, 15, 1134]
                                  ],
             "bought_list": [
                             [156,27],
                             [11,43],
                             [1]
                            ]})

In [119]:
data.apply(lambda x: mrr_at_k(x[1],x[2],5),1).mean()

0.25

## Задание 3*. Реализовать метрику nDCG@k
Normalized discounted cumulative gain. Эту метрику реализовать будет немного сложнее.

$$\Large DCG@K(i) = \sum_{j=1}^{K}\frac{\mathbb{1}_{r_{ij}}}{\log_2 (j+1)}$$


$\Large \mathbb{1}_{r_{ij}}$ -- индикаторная функция показывает что пользователь $i$ провзаимодействовал с продуктом $j$

Для подсчета $nDCG$ нам необходимо найти максимально возможный $DCG$ для пользователя $i$  и рекомендаций длины $K$.
Максимальный $DCG$ достигается когда мы порекомендовали максимально возможное количество релевантных продуктов и все они в начале списка рекомендаций.

$$\Large IDCG@K(i) = max(DCG@K(i)) = \sum_{j=1}^{K}\frac{\mathbb{1}_{j\le|Rel_i|}}{\log_2 (j+1)}$$

$$\Large nDCG@K(i) = \frac {DCG@K(i)}{IDCG@K(i)}$$

$\Large |Rel_i|$ -- количество релевантных продуктов для пользователя $i$



In [120]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #which of the purchases were in Rec.List
    flags = np.isin(recommended_list[:k], bought_list)
    
    dcg_at_k = sum([flags[i]/np.log2(i+2) for i in range(k)])
    idcg_at_k = sum([1/np.log2(i+2) for i in range(k)])
    ndcg_at_k = dcg_at_k / idcg_at_k
    #TODO: Ваш код здесь
    
    return ndcg_at_k #Добавьте сюда результат расчета метрики
    

In [121]:
a = [1,2,3,4,5]
b = [1,2,3]
ndcg_at_k(a,b)

0.7227265726449519

In [122]:
data.apply(lambda x: ndcg_at_k(x[1],x[2],5),1).mean()

0.20748825653397254